In [1]:
import torch as th
from active_critic.learner.active_critic_learner import ActiveCriticLearner, ACLScores
from active_critic.learner.active_critic_args import ActiveCriticLearnerArgs
from active_critic.policy.active_critic_policy import ActiveCriticPolicy
from active_critic.utils.gym_utils import make_vec_env, DummyExtractor, new_epoch_reach, sample_expert_transitions, parse_sampled_transitions
from active_critic.utils.pytorch_utils import build_tf_horizon_mask
from active_critic.utils.dataset import DatasetAC
from active_critic.policy.active_critic_policy import ActiveCriticPolicySetup, ActiveCriticPolicy
from active_critic.model_src.state_model import *
from active_critic.model_src.whole_sequence_model import WholeSequenceModel, WholeSequenceModelArgs
from active_critic.model_src.transformer import ModelSetup


from gym import Env
th.manual_seed(0)

class DummyStateModel(StateModel):
    def __init__(self, args: StateModelArgs) -> None:
        super().__init__(args)

    def forward(self, inpt):
        super().forward(inpt)
        return inpt

def make_wsm_setup(seq_len, d_output, device='cpu'):
    wsm = WholeSequenceModelArgs()
    wsm.model_setup = ModelSetup()
    seq_len = seq_len
    d_output = d_output
    wsm.model_setup.d_output = d_output
    wsm.model_setup.nhead = 1
    wsm.model_setup.d_hid = 200
    wsm.model_setup.d_model = 200
    wsm.model_setup.nlayers = 3
    wsm.model_setup.seq_len = seq_len
    wsm.model_setup.dropout = 0
    wsm.lr = 1e-3
    wsm.model_setup.device = device
    wsm.optimizer_class = th.optim.Adam
    wsm.optimizer_kwargs = {}
    return wsm

def make_acps(seq_len, extractor, new_epoch, batch_size, device, horizon):
    acps = ActiveCriticPolicySetup()
    acps.device=device
    acps.epoch_len=seq_len
    acps.extractor=extractor
    acps.new_epoch=new_epoch
    acps.opt_steps=100
    acps.inference_opt_lr = 1e-2
    acps.optimizer_class = th.optim.SGD
    acps.optimize = True
    acps.batch_size = batch_size
    acps.pred_mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
    acps.opt_mask = th.zeros([seq_len], device=device, dtype=bool)
    acps.opt_mask[-1] = 1
    acps.opt_goal = True
    acps.optimize_goal_emb_acts = False
    acps.goal_label_multiplier = 1
    return acps

def setup_opt_state(batch_size, seq_len, device='cpu'):
    num_cpu = 1
    env, expert = make_vec_env('reach', num_cpu, seq_len=seq_len)
    d_output = env.action_space.shape[0]
    embed_dim = 39
    lr = 5e-4

    actor_args = StateModelArgs()
    actor_args.arch = [512,512, env.action_space.shape[0]]
    actor_args.device = device
    actor_args.lr = lr
    actor = StateModel(args=actor_args)

    critic_args = StateModelArgs()
    critic_args.arch = [512,512, 1]
    critic_args.device = device
    critic_args.lr = lr
    critic = StateModel(args=critic_args)

    inv_critic_args = StateModelArgs()
    inv_critic_args.arch = [200, embed_dim + env.action_space.shape[0]]
    inv_critic_args.device = device
    inv_critic_args.lr = lr
    inv_critic = StateModel(args=inv_critic_args)

    emitter_args = StateModelArgs()
    emitter_args.arch = [200, 200, embed_dim]
    emitter_args.device = device
    emitter_args.lr = lr
    #emitter = StateModel(args=emitter_args)
    emitter = DummyStateModel(args=emitter_args)

    predictor_args = make_wsm_setup(
    seq_len=seq_len, d_output=embed_dim, device=device)
    predictor_args.model_setup.d_hid = 1024
    predictor_args.model_setup.d_model = 1024
    predictor_args.model_setup.nlayers = 4
    predictor_args.model_setup.nhead = 16
    predictor_args.lr = 5e-4
    predictor = WholeSequenceModel(args=predictor_args)

    horizon = seq_len


    acps = make_acps(
        seq_len=seq_len, 
        extractor=DummyExtractor(), 
        new_epoch=new_epoch_reach, 
        device=device, 
        batch_size=batch_size,
        horizon=horizon)
    acps.clip = False
    ac = ActiveCriticPolicy(observation_space=env.observation_space, 
                            action_space=env.action_space,
                            actor=actor,
                            critic=critic,
                            predictor=predictor,
                            emitter=emitter,
                            inverse_critic=inv_critic,
                            acps=acps)
    return ac, acps, batch_size, seq_len, env, expert


def make_acl(device):
    device = device
    acla = ActiveCriticLearnerArgs()
    #acla.data_path = '/home/hendrik/Documents/master_project/LokalData/WSM/'
    acla.data_path = '/data/bing/hendrik/'
    acla.device = device
    acla.extractor = DummyExtractor()
    acla.imitation_phase = False
    acla.logname = 'push data trough 40'
    acla.tboard = True
    acla.batch_size = 32
    acla.validation_episodes = 40
    acla.training_epsiodes = 1
    acla.actor_threshold = 1e-2
    acla.critic_threshold = 1e-3
    acla.predictor_threshold = 1e-2
    acla.gen_scores_threshold = 1
    acla.loss_auto_predictor_threshold = 1e-3
    acla.num_cpu = acla.validation_episodes
    acla.use_pain = True
    acla.patients = 1000
    
    seq_len = 100
    ac, acps, batch_size, seq_len, env, expert= setup_opt_state(device=device, batch_size=acla.batch_size, seq_len=seq_len)
    
    acps.opt_steps = 0
    acla.val_every = 1
    acla.add_data_every = 1

    

    eval_env, expert = make_vec_env('reach', num_cpu=acla.num_cpu, seq_len=seq_len)
    acl = ActiveCriticLearner(ac_policy=ac, env=env, eval_env=eval_env, network_args_obj=acla)
    return acl, env, expert, seq_len, device


/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-14 01:39:34.122156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
acl, env, expert, seq_len, device = make_acl(device='cuda')

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2022-11-14 01:39:40.266589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)

In [3]:
acl.policy.args_obj.goal_label_multiplier

1

In [4]:
acl.train(200)

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Sampling transitions. 2


/home/hendrik/Documents/master_project/Code/active_critic/src/active_critic/utils/gym_utils.py:187: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  actions = th.tensor(actions, dtype=th.float, device=device)


/home/hendrik/Documents/master_project/LokalData/WSM/push data trough 20/best_validation
Success Rate: 0.0
Reward: 0.057259440422058105
training samples: 0
Sampling transitions. 1
Training Reward: tensor([[0.0554]], device='cuda:0')█████████████████████████████████████████████████-| 99.0% Predicting Epsiode


/home/hendrik/Documents/master_project/Code/active_critic/src/active_critic/learner/active_critic_learner.py:165: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.write_tboard_scalar({'Pain': th.tensor(pain)}, train=True, step=self.pain_step)


Sampling transitions. 1
Training Reward: tensor([[0.0277]], device='cuda:0')█████████████████████████████████████████████████-| 99.0% Predicting Epsiode
Sampling transitions. 2
Success Rate: 0.0████████████████████████████████████████████████████████████████████████████████████-| 99.0% Predicting Epsiode
Reward: 0.040860578417778015
training samples: 2
Sampling transitions. 1
Training Reward: tensor([[0.0309]], device='cuda:0')█████████████████████████████████████████████████-| 99.0% Predicting Epsiode
Sampling transitions. 2
Success Rate: 0.0████████████████████████████████████████████████████████████████████████████████████-| 99.0% Predicting Epsiode
Reward: 0.03123351000249386
training samples: 3
Sampling transitions. 1
Training Reward: tensor([[0.0276]], device='cuda:0')█████████████████████████████████████████████████-| 99.0% Predicting Epsiode
Sampling transitions. 2
Success Rate: 0.0████████████████████████████████████████████████████████████████████████████████████-| 99.0% Pred

In [ ]:
seq_len

In [ ]:
eval_env, expert = make_vec_env('reach', num_cpu=1, seq_len=seq_len)

In [ ]:
acl.policy.reset()

In [ ]:
obs = eval_env.reset()
action = acl.policy.predict(obs)

In [ ]:
acl.policy.history.scores[0].shape

In [ ]:
acl.policy.history.emb[0].shape

In [ ]:
embeddings, actions = acl.policy.build_sequence(
                embeddings=acl.policy.current_embeddings, 
                actions=actions[:,:0], 
                seq_len=10, 
                goal_emb_acts=acl.policy.goal_emb_acts,
                goal_state=None,
                tf_mask=acl.policy.args_obj.pred_mask,
                actor=acl.policy.actor,
                predictor=acl.policy.predicor)

In [ ]:
actions.shape

In [ ]:
acl.policy.eval()
acl.policy.reset()
seq_len = acl.policy.args_obj.epoch_len
transitions = sample_expert_transitions(
    acl.policy.predict, eval_env, 1)

In [ ]:
actions, observations, rewards = parse_sampled_transitions(
            transitions=transitions, extractor=acl.policy.args_obj.extractor, device=device)

In [ ]:
embeddings = acl.policy.emitter.forward(observations)

In [ ]:
critic_input = acl.policy.get_critic_input(embeddings=embeddings, actions=actions)

In [ ]:
scores = acl.policy.critic.forward(critic_input)

In [ ]:
calcMSE(scores, rewards)

In [ ]:
rewards

In [ ]:
observations.shape

In [ ]:
acl.policy.history.emb[0][:,0,-1].shape

In [ ]:
eval_env, expert = make_vec_env('reach', num_cpu=1, seq_len=seq_len)
acl.policy.reset()
obs = eval_env.reset()
action = acl.policy.predict(obs)

In [ ]:
embeddings, actions = acl.policy.build_sequence(
    embeddings=acl.policy.current_embeddings, 
    actions=None, 
    seq_len=seq_len, 
    goal_emb_acts=acl.policy.goal_emb_acts,
    goal_state=None,
    tf_mask=acl.policy.args_obj.pred_mask,
    actor=acl.policy.actor,
    predictor=acl.policy.predicor)

In [ ]:
embeddings.shape

In [ ]:
scores = acl.policy.critic.forward(embeddings)

In [ ]:
current_step = 49
predicotr_input = acl.policy.get_predictor_input(embeddings[:,:current_step], actions[:,:current_step])

In [ ]:
next_states, attention = acl.policy.predicor.model.forward(src=predicotr_input, mask = acl.policy.args_obj.pred_mask[:current_step,:current_step], return_attention=True)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel('Query')
ax.set_xlabel('Key')
cax = ax.matshow(attention[0].to('cpu'), interpolation='nearest')
fig.colorbar(cax)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel('Query')
ax.set_xlabel('Key')
cax = ax.matshow(attention[0].to('cpu'), interpolation='nearest')
fig.colorbar(cax)
plt.show()